# Importação das bibliotecas necessárias

In [5]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [6]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPDadosAbertosReceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-17 10:01:28,265 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [7]:
df_compliance_empresas  = extrator_bronze.criar_view_temporaria('bronze/dadosabertosreceita/empresas','compliance_receita_empresas')

2024-10-17 10:01:28,964 - INFO - Aguarde... Criando View (bronze/dadosabertosreceita/empresas)
2024-10-17 10:01:34,740 - INFO - View criada com sucesso


In [14]:
from pyspark.sql.functions import col
df_compliance_empresas.select('*').where(col('cnpj_basico') == '19092936').show()

+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsavel|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|   19092936|FIONOBRE DIAMOND ...|             2062|                      49|     500000,00|           03|                       NULL|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+



# Tratamentos na tabela 

In [4]:
sql_query = """
SELECT
    cnpj_basico AS cd_cnpj_basico,
    razao_social AS nm_razao_social,
    CASE 
        WHEN porte_empresa = '00' THEN 'NÃO INFORMADO'
        WHEN porte_empresa = '01' THEN 'MICRO EMPRESA'
        WHEN porte_empresa = '03' THEN 'EMPRESA DE PEQUENO PORTE'
        WHEN porte_empresa = '05' THEN 'DEMAIS'
        ELSE 'Outros'
    END AS dsc_porte_empresa
FROM compliance_receita_empresas
"""

df_compliance_empresas  = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 18:57:13,374 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 18:57:13,375 - INFO - 
SELECT
    cnpj_basico AS cd_cnpj_basico,
    razao_social AS nm_razao_social,
    CASE 
        WHEN porte_empresa = '00' THEN 'NÃO INFORMADO'
        WHEN porte_empresa = '01' THEN 'MICRO EMPRESA'
        WHEN porte_empresa = '03' THEN 'EMPRESA DE PEQUENO PORTE'
        WHEN porte_empresa = '05' THEN 'DEMAIS'
        ELSE 'Outros'
    END AS dsc_porte_empresa
FROM compliance_receita_empresas

2024-09-25 18:57:13,477 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [5]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/DadosAbertosReceita/DadosAbertosReceitaEmpresas'

In [6]:
extrator_bronze.salvar_delta(df_compliance_empresas, 'overwrite')

2024-09-25 18:57:13,492 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 18:57:48,981 - INFO - Dados persistidos com sucesso
2024-09-25 18:57:48,981 - INFO - s3a://silver/DadosAbertosReceita/DadosAbertosReceitaEmpresas
2024-09-25 18:57:48,982 - INFO - Aguarde... Realizando optimize
2024-09-25 18:59:05,347 - INFO - Optimize executado com sucesso.
2024-09-25 18:59:05,347 - INFO - Aguarde... Realizando vacuum
2024-09-25 18:59:21,714 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [7]:
extrator_bronze.parar_sessao()

2024-09-25 18:59:22,265 - INFO - Sessão Spark finalizada.
